# Modelling & Analysis of Gas Turbine Blade Behavior to Predict Turbine Blade Resonance Amplitudes

# NEURAL NETWORK MODEL

IMPORTING LIBRARIES

In [2]:
import tensorflow as tf
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten
from sklearn.model_selection import cross_val_score

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import os.path as path

from keras.optimizers import Adam
from keras import backend as K

Using TensorFlow backend.


MAKING A SESIION IN KERAS

In [2]:
config = tf.ConfigProto( device_count = {'GPU': 1})
#sess = tf.Session(config=config) 
sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True,log_device_placement=True))
keras.backend.set_session(sess)

Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device



LOADING DATASETS

In [3]:
train = pd.read_csv("traindata.csv")
train=train.sample(frac=1)
print(np.shape(train))
train.head()

(143, 8)


,N,m,P1,T1,P3,T3,TJC,A
69,98.6,55.7,13.1168,19.5,249.3,725.5,690.7,29.53
96,98.6,75.3,13.1168,19.5,211.6,725.5,637.7,45.00
121,98.6,75.3,13.2544,27.0,211.6,725.5,690.7,99.23
27,94.8,55.7,13.2544,27.0,211.6,761.0,690.7,34.73
49,94.8,75.3,13.2544,19.5,211.6,725.5,690.7,63.00


LOADING VALIDATION DATA

In [4]:
validate = pd.read_csv("validatedata.csv")
#validate = validate.sample(frac=1)
validate=validate.rename(columns={"Amplitude (Max)": "A"})
print(np.shape(validate))
validate.head()

(43, 8)


,N,m,P1,T1,P3,T3,TJC,A
0,95.84,63.78,13.16,19.59,216.63,735.61,667.44,42.9156
1,97.47,56.41,13.13,22.03,236.53,759.09,683.22,45.2331
2,96.18,64.27,13.21,26.66,225.33,759.51,673.38,65.4893
3,95.90,62.89,13.21,25.73,227.34,732.96,673.73,60.4748
4,94.88,56.32,13.15,20.78,236.98,744.53,669.48,20.7956


APPENDING BOTH TRAINING AND VALIDATION DATASET

In [5]:
train.append(validate,ignore_index=True)

,N,m,P1,T1,P3,T3,TJC,A
0,98.60,55.70,13.1168,19.50,249.30,725.50,690.70,29.5300
1,98.60,75.30,13.1168,19.50,211.60,725.50,637.70,45.0000
2,98.60,75.30,13.2544,27.00,211.60,725.50,690.70,99.2300
3,94.80,55.70,13.2544,27.00,211.60,761.00,690.70,34.7300
4,94.80,75.30,13.2544,19.50,211.60,725.50,690.70,63.0000
...,...,...,...,...,...,...,...,...
181,95.44,72.47,13.2200,20.49,237.12,760.38,686.85,82.7935
182,96.35,57.40,13.1300,26.56,224.70,753.34,646.88,42.6216
183,98.24,62.05,13.1700,20.55,224.12,759.24,642.68,58.6157
184,96.73,69.60,13.1600,24.35,224.97,747.12,688.88,83.8713


In [6]:
train['T3']=train['T3']/10
train['TJC']=train['TJC']/10

LOADING TEST DATA

In [7]:
test = pd.read_csv("testdata.csv")
test=test.rename(columns={"NL": "N","Analytical value":"A","TJ":"TJC"})
del test['Experimental value']
print(np.shape(test))
test.head()


(24, 8)


,N,m,P1,T1,P3,T3,TJC,A
0,95.0504,69.8002,13.13,25.8,222.5,752.0,685.0,76.0174
1,95.7920,71.5995,13.16,26.1,233.5,742.9,651.4,72.6498
2,96.1042,72.1003,13.12,26.1,235.2,743.5,646.3,68.9307
3,96.4946,72.4002,13.15,24.5,234.3,761.0,686.9,67.4881
4,97.3730,74.1005,13.15,25.2,243.9,747.9,672.0,68.2048


MAKING ALL INPUT VARIABLES IN RANGE OF 0-1, FOR EASY COMPUTATION

In [8]:
test['T3']=validate['T3']/10
test['TJC']=validate['TJC']/10

SPLITTING INTO TRAINING AND TESTING

In [9]:
features = train.columns[:7]
X_train = train[features]/100
y_train = train['A']/100
X_train=np.asarray(X_train)
y_train=np.asarray(y_train)
features = test.columns[:7]
X_test = test[features]/100
y_test = test['A']/100

In [10]:
X_train

array([[0.986   , 0.557   , 0.131168, ..., 2.493   , 0.7255  , 0.6907  ],
       [0.986   , 0.753   , 0.131168, ..., 2.116   , 0.7255  , 0.6377  ],
       [0.986   , 0.753   , 0.132544, ..., 2.116   , 0.7255  , 0.6907  ],
       ...,
       [0.986   , 0.753   , 0.132544, ..., 2.304   , 0.761   , 0.6907  ],
       [0.948   , 0.557   , 0.131168, ..., 2.116   , 0.7255  , 0.6377  ],
       [0.948   , 0.753   , 0.131168, ..., 2.116   , 0.761   , 0.6377  ]])

In [11]:
y_train

array([0.2953, 0.45  , 0.9923, 0.3473, 0.63  , 0.5906, 0.9767, 0.315 ,
       0.3101, 0.8682, 0.4134, 0.63  , 0.7875, 0.4725, 0.4961, 0.2481,
       0.2171, 1.1813, 0.945 , 0.3938, 0.3   , 0.8269, 0.6946, 0.4725,
       0.2171, 0.1538, 0.225 , 0.9923, 0.7875, 0.3   , 0.315 , 0.4961,
       1.2403, 0.3308, 0.5906, 0.3256, 0.2067, 1.0853, 0.4725, 0.4725,
       0.1538, 0.6512, 0.3308, 1.2403, 0.3308, 0.75  , 0.945 , 0.1969,
       0.315 , 0.6202, 0.6202, 0.4341, 0.3308, 0.8682, 0.3473, 0.945 ,
       0.1969, 0.375 , 0.5091, 0.45  , 0.5906, 0.1575, 0.3101, 0.2953,
       0.1688, 0.2363, 0.1654, 0.4961, 0.3938, 1.2713, 0.1969, 0.6202,
       0.4341, 0.2481, 0.5209, 1.1813, 0.5209, 0.4341, 0.1736, 0.6512,
       0.3308, 0.15  , 0.8269, 0.225 , 0.4961, 0.3308, 0.6202, 0.852 ,
       0.2363, 0.2067, 0.315 , 0.3256, 0.4961, 0.1875, 0.2953, 0.7262,
       0.63  , 0.9923, 0.6512, 0.2813, 0.315 , 1.125 , 0.2605, 1.3023,
       0.5625, 0.5906, 0.4134, 0.1654, 0.9   , 0.7875, 1.0419, 0.6   ,
      

In [27]:
X_test

,N,m,P1,T1,P3,T3,TJC
0,0.950504,0.698002,0.1313,0.258,2.225,0.73561,0.66744
1,0.957920,0.715995,0.1316,0.261,2.335,0.75909,0.68322
2,0.961042,0.721003,0.1312,0.261,2.352,0.75951,0.67338
3,0.964946,0.724002,0.1315,0.245,2.343,0.73296,0.67373
4,0.973730,0.741005,0.1315,0.252,2.439,0.74453,0.66948
5,0.961045,0.718004,0.1314,0.270,2.332,0.75186,0.64378
6,0.964399,0.728000,0.1319,0.244,2.391,0.76030,0.66631
7,0.965030,0.739995,0.1316,0.266,2.441,0.73043,0.67533
8,0.963303,0.725001,0.1320,0.238,2.380,0.74314,0.67765
9,0.968761,0.737996,0.1320,0.232,2.434,0.75434,0.64282


NEURAL NETWORK WITH DIFFERENT VARYING PARAMETERS

In [13]:
#NO. OF HIDDEN LAYERS
#NO. OF NEURONS IN EACH LAYER
#NO. OF TIMES ITERATION
#BATCH SIZE

In [14]:
model = Sequential()
model.add(Dense(128, input_dim=7, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# Fit the model
model.fit(X_train, y_train, validation_split=0.20, epochs=420, batch_size=10, verbose=0)
Y_pred= model.predict(X_test)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



In [6]:
# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")

Loaded model from disk


In [3]:
loaded_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 128)               1024      
_________________________________________________________________
dense_2 (Dense)              (None, 32)                4128      
_________________________________________________________________
dense_3 (Dense)              (None, 8)                 264       
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 9         
Total params: 5,425
Trainable params: 5,425
Non-trainable params: 0
_________________________________________________________________


Hence our model is loaded and saved as model.h5